IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M3\NO_FAULT3


In [3]:
guasto=0

VOLO m3 NO FAULT

In [4]:
rcou_m3_nofault = pd.read_csv("RCOU.csv")
rcou_m3_nofault = rcou_m3_nofault.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m3_nofault = rcou_m3_nofault[((rcou_m3_nofault['C9'] >= potenza) & (rcou_m3_nofault['C10'] >= potenza) & (rcou_m3_nofault['C11'] >= potenza) & (rcou_m3_nofault['C12'] >= potenza) & (rcou_m3_nofault['C13'] >= potenza) & (rcou_m3_nofault['C14']>= potenza))]
rcou_m3_nofault=rcou_m3_nofault.reset_index(drop=True)
display(rcou_m3_nofault)

,TimeUS,C9,C10,C11,C12,C13,C14
0,85323839,1415,1364,1394,1387,1416,1363
1,85345385,1411,1395,1437,1368,1441,1363
2,85356318,1376,1441,1404,1415,1366,1450
3,85367015,1409,1423,1421,1411,1412,1420
4,85423541,1523,1364,1442,1456,1515,1375
...,...,...,...,...,...,...,...
6615,162729348,1551,1519,1559,1510,1447,1613
6616,162740384,1598,1461,1520,1545,1460,1599
6617,162751646,1620,1423,1544,1513,1512,1545
6618,162761906,1593,1437,1553,1485,1496,1543


In [5]:
min=rcou_m3_nofault['TimeUS'][0]
max=rcou_m3_nofault['TimeUS'][len(rcou_m3_nofault)-1]
print(max)
print(min)

162772643
85323839


In [6]:
xkf1_m3_nofault = pd.read_csv("XKF1_0.csv")
xkf1_m3_nofault = xkf1_m3_nofault.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m3_nofault = xkf1_m3_nofault[((xkf1_m3_nofault['TimeUS'] >= min) & (xkf1_m3_nofault['TimeUS'] <= max))]
xkf1_m3_nofault = xkf1_m3_nofault.reset_index(drop=True)
print(xkf1_m3_nofault)

         TimeUS  Roll  Pitch     Yaw
0      85332681  1.98  -0.84    3.31
1      85343445  1.98  -0.88    3.31
2      85353408  1.91  -0.83    3.33
3      85364246  1.93  -0.80    3.33
4      85375369  2.02  -0.82    3.32
...         ...   ...    ...     ...
7005  162727101 -1.18   1.24  358.17
7006  162738085 -1.06   1.13  358.20
7007  162749622 -0.97   1.00  358.25
7008  162759649 -0.91   0.94  358.31
7009  162770128 -0.79   0.88  358.39

[7010 rows x 4 columns]


In [7]:
att_m3_nofault = pd.read_csv("ATT.csv")
att_m3_nofault = att_m3_nofault.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m3_nofault = att_m3_nofault[((att_m3_nofault['TimeUS'] >= min) & (att_m3_nofault['TimeUS'] <= max	))]
att_m3_nofault=att_m3_nofault.reset_index(drop=True)
print(att_m3_nofault)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0      85334020     0.12  1.98      0.24  -0.84    3.31    3.31
1      85344873     0.12  1.98      0.25  -0.88    3.31    3.31
2      85355543     0.12  1.91      0.26  -0.83    3.31    3.33
3      85366178     0.12  1.93      0.26  -0.80    3.31    3.33
4      85377132     0.12  2.02      0.25  -0.82    3.31    3.32
...         ...      ...   ...       ...    ...     ...     ...
7007  162728615     0.96 -1.18     -0.36   1.24  359.91  358.18
7008  162739678     0.95 -1.06     -0.37   1.13  359.91  358.21
7009  162751006     0.93 -0.97     -0.38   1.00  359.91  358.26
7010  162761104     0.89 -0.91     -0.36   0.94  359.91  358.32
7011  162771993     0.83 -0.79     -0.33   0.88  359.91  358.40

[7012 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m3_nofault)>len(xkf1_m3_nofault):
    to_add=att_m3_nofault[len(xkf1_m3_nofault):]
    att_m3_nofault=att_m3_nofault[:len(xkf1_m3_nofault)]

if len(xkf1_m3_nofault)>len(att_m3_nofault):
    to_add=xkf1_m3_nofault[len(att_m3_nofault):]

for idx,i in enumerate(att_m3_nofault['Roll']):
    if(xkf1_m3_nofault['Roll'][idx] != i):
        att_m3_nofault['Roll'][idx] = ((att_m3_nofault['Roll'][idx] + xkf1_m3_nofault['Roll'][idx])/2)

for idx,i in enumerate(att_m3_nofault['Pitch']):
    if(xkf1_m3_nofault['Pitch'][idx] != i):
        att_m3_nofault['Pitch'][idx] = ((att_m3_nofault['Pitch'][idx] + xkf1_m3_nofault['Pitch'][idx])/2)

for idx,i in enumerate(att_m3_nofault['Yaw']):
    if(xkf1_m3_nofault['Yaw'][idx] != i):
        att_m3_nofault['Yaw'][idx] = ((att_m3_nofault['Yaw'][idx] + xkf1_m3_nofault['Yaw'][idx])/2)

if not to_add.empty:
    att_m3_nofault=pd.concat([att_m3_nofault,to_add])

print(att_m3_nofault)

C:\Users\mgale\AppData\Local\Temp\ipykernel_5400\1419942214.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m3_nofault['Roll'][idx] = ((att_m3_nofault['Roll'][idx] + xkf1_m3_nofault['Roll'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_5400\1419942214.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m3_nofault['Pitch'][idx] = ((att_m3_nofault['Pitch'][idx] + xkf1_m3_nofault['Pitch'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_5400\1419942214.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

         TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw      Yaw
0      85334020     0.12  1.980      0.24 -0.840    3.31    3.310
1      85344873     0.12  1.980      0.25 -0.880    3.31    3.310
2      85355543     0.12  1.910      0.26 -0.830    3.31    3.330
3      85366178     0.12  1.930      0.26 -0.800    3.31    3.330
4      85377132     0.12  2.020      0.25 -0.820    3.31    3.320
...         ...      ...    ...       ...    ...     ...      ...
7007  162728615     0.96 -1.075     -0.36  1.120  359.91  358.215
7008  162739678     0.95 -0.985     -0.37  1.035  359.91  358.260
7009  162751006     0.93 -0.880     -0.38  0.940  359.91  358.325
7010  162761104     0.89 -0.910     -0.36  0.940  359.91  358.320
7011  162771993     0.83 -0.790     -0.33  0.880  359.91  358.400

[7012 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m3_nofault = pd.read_csv("ESC_0.csv")
esc_0_m3_nofault = esc_0_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m3_nofault = esc_0_m3_nofault[((esc_0_m3_nofault['TimeUS'] >= min) & (esc_0_m3_nofault['TimeUS'] <= max))]
esc_0_m3_nofault=esc_0_m3_nofault.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m3_nofault = pd.read_csv("ESC_1.csv")
esc_1_m3_nofault = esc_1_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m3_nofault = esc_1_m3_nofault[((esc_1_m3_nofault['TimeUS'] >= min) & (esc_1_m3_nofault['TimeUS'] <= max))]
esc_1_m3_nofault=esc_1_m3_nofault.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m3_nofault = pd.read_csv("ESC_2.csv")
esc_2_m3_nofault = esc_2_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m3_nofault = esc_2_m3_nofault[((esc_2_m3_nofault['TimeUS'] >= min) & (esc_2_m3_nofault['TimeUS'] <= max))]
esc_2_m3_nofault=esc_2_m3_nofault.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m3_nofault = pd.read_csv("ESC_3.csv")
esc_3_m3_nofault = esc_3_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m3_nofault = esc_3_m3_nofault[((esc_3_m3_nofault['TimeUS'] >= min) & (esc_3_m3_nofault['TimeUS'] <= max))]
esc_3_m3_nofault=esc_3_m3_nofault.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m3_nofault = pd.read_csv("ESC_4.csv")
esc_4_m3_nofault = esc_4_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m3_nofault = esc_4_m3_nofault[((esc_4_m3_nofault['TimeUS'] >= min) & (esc_4_m3_nofault['TimeUS'] <= max))]
esc_4_m3_nofault=esc_4_m3_nofault.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m3_nofault = pd.read_csv("ESC_5.csv")
esc_5_m3_nofault = esc_5_m3_nofault.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m3_nofault = esc_5_m3_nofault[((esc_5_m3_nofault['TimeUS'] >= min) & (esc_5_m3_nofault['TimeUS'] <= max))]
esc_5_m3_nofault=esc_5_m3_nofault.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m3_nofault = pd.read_csv("IMU_0.csv")
imu_0_m3_nofault = imu_0_m3_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m3_nofault = imu_0_m3_nofault[((imu_0_m3_nofault['TimeUS'] >= min) & (imu_0_m3_nofault['TimeUS'] <= max))]

imu_1_m3_nofault = pd.read_csv("IMU_1.csv")
imu_1_m3_nofault = imu_1_m3_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m3_nofault = imu_1_m3_nofault[((imu_1_m3_nofault['TimeUS'] >= min) & (imu_1_m3_nofault['TimeUS'] <= max))]

imu_2_m3_nofault = pd.read_csv("IMU_2.csv")
imu_2_m3_nofault = imu_2_m3_nofault.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m3_nofault = imu_2_m3_nofault[((imu_2_m3_nofault['TimeUS'] >= min) & (imu_2_m3_nofault['TimeUS'] <= max))]

imu_m3_nofault = pd.concat((imu_0_m3_nofault, imu_1_m3_nofault, imu_2_m3_nofault))
imu_m3_nofault=imu_m3_nofault.groupby(imu_m3_nofault.TimeUS, as_index=False).mean()

display(imu_m3_nofault)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,85325609,0.027300,0.059992,0.035757,-0.274592,-0.153920,-9.882955
1,85328405,0.010440,0.088152,0.048246,-0.163496,-0.248711,-9.751685
2,85333916,-0.054220,-0.096859,-0.017186,-0.235180,-0.172751,-9.537430
3,85339625,0.009177,0.034897,0.022398,-0.238212,-0.308036,-10.156127
4,85342546,-0.040668,0.042636,0.027003,-0.262722,-0.363957,-10.009839
...,...,...,...,...,...,...,...
26643,162760938,0.072825,-0.033702,0.107191,1.234315,1.531393,-12.105240
26644,162763425,0.250181,-0.223798,0.084430,1.207410,1.538579,-12.511827
26645,162766175,0.185225,-0.168557,0.087139,0.760958,1.467667,-12.682097
26646,162768911,0.051801,0.000553,0.112503,0.974692,1.827939,-12.768413


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m3_nofault = pd.merge_ordered(imu_m3_nofault,att_m3_nofault)
m3_nofault=m3_nofault.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m3_nofault = pd.merge_ordered(m3_nofault, esc_0_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_1_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_2_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_3_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_4_m3_nofault)
m3_nofault = pd.merge_ordered(m3_nofault, esc_5_m3_nofault)
m3_nofault=m3_nofault.fillna(method="ffill").fillna(method="bfill")
m3_nofault = pd.merge_ordered(m3_nofault, rcou_m3_nofault)
m3_nofault=m3_nofault.fillna(method="ffill").fillna(method="bfill")

In [12]:
m3_nofault["TimeUS"] = m3_nofault["TimeUS"] - m3_nofault.iloc[0]["TimeUS"]
m3_nofault["TimeUS"] = m3_nofault["TimeUS"].astype(int)
m3_nofault["Guasto"] = guasto

In [13]:
last_cell = m3_nofault.iloc[-1, m3_nofault.columns.get_loc('TimeUS')]
print(last_cell)

77448804


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
27104  77436128
27105  77438985
27106  77441842
27107  77444699
27108  77447556

[27109 rows x 1 columns]


In [16]:
m3_nofault_final = pd.merge_ordered(m3_nofault,df)
m3_nofault_final=m3_nofault_final.fillna(method="ffill").fillna(method="bfill")
print(m3_nofault_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0  0.027300  0.059992  0.035757 -0.274592 -0.153920  -9.882955   
1          1770  0.027300  0.059992  0.035757 -0.274592 -0.153920  -9.882955   
2          2857  0.027300  0.059992  0.035757 -0.274592 -0.153920  -9.882955   
3          4566  0.010440  0.088152  0.048246 -0.163496 -0.248711  -9.751685   
4          5714  0.010440  0.088152  0.048246 -0.163496 -0.248711  -9.751685   
...         ...       ...       ...       ...       ...       ...        ...   
71298  77445310  0.051801  0.000553  0.112503  0.974692  1.827939 -12.768413   
71299  77447556  0.051801  0.000553  0.112503  0.974692  1.827939 -12.768413   
71300  77447761  0.081295 -0.020807  0.123744  1.362946  2.211410 -12.852490   
71301  77448154  0.081295 -0.020807  0.123744  1.362946  2.211410 -12.852490   
71302  77448804  0.081295 -0.020807  0.123744  1.362946  2.211410 -12.852490   

       DesRoll  Roll  DesPitch  ...  Cu

In [17]:
m3_nofault_final = m3_nofault_final[m3_nofault_final.TimeUS.isin(to_be)]

In [18]:
m3_nofault_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,0.027300,0.059992,0.035757,-0.274592,-0.153920,-9.882955,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
2,2857,0.027300,0.059992,0.035757,-0.274592,-0.153920,-9.882955,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
4,5714,0.010440,0.088152,0.048246,-0.163496,-0.248711,-9.751685,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
5,8571,0.010440,0.088152,0.048246,-0.163496,-0.248711,-9.751685,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
9,11428,-0.054220,-0.096859,-0.017186,-0.235180,-0.172751,-9.537430,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71288,77436128,0.034045,-0.006684,0.107095,0.722516,1.277058,-11.917500,0.93,-0.88,-0.38,...,2.55,5545.0,2.77,1620.0,1423.0,1544.0,1513.0,1512.0,1545.0,0.0
71292,77438985,0.072825,-0.033702,0.107191,1.234315,1.531393,-12.105240,0.89,-0.91,-0.36,...,2.55,5545.0,2.77,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0.0
71294,77441842,0.250181,-0.223798,0.084430,1.207410,1.538579,-12.511827,0.89,-0.91,-0.36,...,2.55,5545.0,2.77,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0.0
71296,77444699,0.185225,-0.168557,0.087139,0.760958,1.467667,-12.682097,0.89,-0.91,-0.36,...,2.55,5545.0,2.77,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0.0


In [19]:
from datetime import timedelta
m3_nofault_final=m3_nofault_final.reset_index(drop=True)
m3_nofault_final['TimeUS'] = pd.to_datetime(m3_nofault_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m3_nofault_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.027300,0.059992,0.035757,-0.274592,-0.153920,-9.882955,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
1,00:00:00.002857,0.027300,0.059992,0.035757,-0.274592,-0.153920,-9.882955,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
2,00:00:00.005714,0.010440,0.088152,0.048246,-0.163496,-0.248711,-9.751685,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
3,00:00:00.008571,0.010440,0.088152,0.048246,-0.163496,-0.248711,-9.751685,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
4,00:00:00.011428,-0.054220,-0.096859,-0.017186,-0.235180,-0.172751,-9.537430,0.12,1.98,0.24,...,2.63,3065.0,1.60,1415.0,1364.0,1394.0,1387.0,1416.0,1363.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27104,00:01:17.436128,0.034045,-0.006684,0.107095,0.722516,1.277058,-11.917500,0.93,-0.88,-0.38,...,2.55,5545.0,2.77,1620.0,1423.0,1544.0,1513.0,1512.0,1545.0,0.0
27105,00:01:17.438985,0.072825,-0.033702,0.107191,1.234315,1.531393,-12.105240,0.89,-0.91,-0.36,...,2.55,5545.0,2.77,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0.0
27106,00:01:17.441842,0.250181,-0.223798,0.084430,1.207410,1.538579,-12.511827,0.89,-0.91,-0.36,...,2.55,5545.0,2.77,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0.0
27107,00:01:17.444699,0.185225,-0.168557,0.087139,0.760958,1.467667,-12.682097,0.89,-0.91,-0.36,...,2.55,5545.0,2.77,1593.0,1437.0,1553.0,1485.0,1496.0,1543.0,0.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m3_nofault_final)/350)):
        V1=m3_nofault_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_5400\630337658.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_5400\630337658.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Ca

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.062662,0.012958,0.155209,0.129797,0.0,66.0,11.661436,5.406596,0.026618,...,9120.646103,3.507066,1.291774,-0.975520,3.686136,115.0,114.0,17.226055,13.471534,0
1,1.0,-0.062530,0.012927,0.160190,0.129615,0.0,65.0,11.768925,4.692885,0.026006,...,8967.680827,3.508722,1.289059,-0.967724,3.687346,107.0,106.0,16.936737,12.818853,0
2,2.0,-0.062674,0.012897,0.166844,0.129572,0.0,66.0,11.777719,4.484673,0.025690,...,8802.870106,3.510368,1.286354,-0.959914,3.688549,117.0,116.0,16.632712,12.159002,0
3,3.0,-0.062557,0.012866,0.172529,0.129393,0.0,63.0,11.877089,4.982702,0.025167,...,8624.634920,3.512006,1.283659,-0.952090,3.689745,103.0,102.0,16.315747,11.498793,0
4,4.0,-0.062441,0.012834,0.178231,0.129216,0.0,67.0,11.975022,5.459079,0.024647,...,8431.644832,3.513634,1.280974,-0.944253,3.690933,107.0,106.0,15.987575,10.844545,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,72.0,-0.066922,0.011219,0.563032,0.125183,0.0,89.0,11.660220,5.020573,0.011211,...,11596.317962,3.633522,1.153887,-0.409675,3.788620,132.0,133.0,20.494576,20.368047,0
73,73.0,-0.066955,0.011193,0.571572,0.125097,0.0,90.0,11.646088,5.322145,0.010899,...,11391.342284,3.635354,1.152581,-0.401773,3.790206,117.0,116.0,20.410529,20.272605,0
74,74.0,-0.067011,0.011168,0.579857,0.125027,0.0,88.0,11.622338,5.116798,0.010713,...,11175.266390,3.637176,1.151275,-0.393862,3.791783,111.0,110.0,20.440144,20.019788,0
75,75.0,-0.067020,0.011141,0.588374,0.124927,0.0,84.0,11.617996,4.856946,0.010561,...,10950.830794,3.638991,1.149968,-0.385942,3.793352,115.0,114.0,20.456962,19.738207,0


In [23]:
path_file = path_file.replace(r"M3\NO_FAULT3", "")
os.chdir(path_file)
df_merged.to_csv('m3_nofault.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
